In [ ]:
from litellm import completion
import json
import strands, strands_tools

# Model configuration - using AWS Bedrock inference profile
MODEL = "bedrock/arn:aws:bedrock:us-east-2:195699526104:inference-profile/us.anthropic.claude-3-5-haiku-20241022-v1:0"

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful agents that explains things in simple language"
    },
    {
        "role": "user",
        "content": "What's recursion?"
    }
]

In [ ]:
response = completion(model=MODEL, messages=messages)

In [ ]:
print (response.choices[0].message.content)

In [ ]:
# Define tools the LLM can use (JSON Schema format)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g., 'San Francisco, CA'",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "Temperature unit",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Madison, WI"}]

response = completion(
    model=MODEL,
    messages = messages,
    tools = tools,
)

In [ ]:
print(response.choices[0].message)

In [ ]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Perform a mathematical calculation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "Math expression to evaluate, e.g., '2 + 2 * 3'",
                        }
                    },
                    "required": ["expression"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_exchange_rate",
                "description": "Get the exchange rate between two currencies",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "from_currency": {
                            "type": "string",
                            "description": "Source currency code, e.g., 'USD'",
                        },
                        "to_currency": {
                            "type": "string",
                            "description": "Target currency code, e.g., 'EUR'",
                        },
                    },
                    "required": ["from_currency", "to_currency"],
                },
            },
        },
    ]

In [ ]:
# Simulated tool implementations
def execute_tool(name: str, arguments: dict) -> str:
    """Execute a tool and return the result as a string."""
    if name == "calculate":
        try:
            # WARNING: eval is dangerous!
            result = eval(arguments["expression"])
            return json.dumps({"result": result})
        except Exception as e:
            return json.dumps({"error": str(e)})

    elif name == "get_exchange_rate":
        # Simulated exchange rates
        rates = {
            ("USD", "EUR"): 0.92,
            ("USD", "JPY"): 149.50,
            ("EUR", "USD"): 1.09,
        }
        key = (arguments["from_currency"], arguments["to_currency"])
        rate = rates.get(key, 1.0)
        return json.dumps(
            {
                "from": arguments["from_currency"],
                "to": arguments["to_currency"],
                "rate": rate,
            }
        )

    return json.dumps({"error": "Unknown tool"})


In [ ]:
 # Start the conversation
messages = [
    {
        "role": "user",
        "content": "I have 150 USD. How much is that in EUR? Then calculate what 15% tip would be on that EUR amount.",
    }
]

In [ ]:
print(f"\nUser: {messages[0]['content']}")
print("\n--- Agentic Loop Starting ---")

# The agentic loop
for iteration in range(1, 11):
    print(f"\n[Iteration {iteration}]")

    response = completion(model=MODEL, messages=messages, tools=tools)
    assistant_message = response.choices[0].message
    messages.append(assistant_message.model_dump())

    if not assistant_message.tool_calls:
        print("\n--- Agentic Loop Complete ---")
        print(f"\nFinal Answer: {assistant_message.content}")
        break

    for tc in assistant_message.tool_calls:
        args = json.loads(tc.function.arguments)
        print(f"  Tool call: {tc.function.name}({args})")
        result = execute_tool(tc.function.name, args)
        print(f"  Result: {result}")
        messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})
else:
    print("\nWarning: Max iterations reached!")

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "bash",
            "description": "Run a bash command and return stdout/stderr",
            "parameters": {
                "type": "object",
                "properties": {
                    "command": {"type": "string", "description": "The bash command to run"}
                },
                "required": ["command"],
            },
        },
    }
]


import subprocess
def run_bash(command: str) -> str:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout + result.stderr or "(no output)"

messages = [
    {"role": "system", "content": "You are a helpful assistant that can run bash commands. Be concise."},
    {"role": "user", "content": "List the Python files in the current directory and count them."},
]

print(f"\nUser: {messages[1]['content']}")

for _ in range(10):
    response = completion(model=MODEL, messages=messages, tools=tools)
    msg = response.choices[0].message
    messages.append(msg.model_dump())

    if not msg.tool_calls:
        print(f"\nAssistant: {msg.content}")
        break

    for tc in msg.tool_calls:
        cmd = json.loads(tc.function.arguments)["command"]
        print(f"\n$ {cmd}")
        output = run_bash(cmd)
        print(output)
        messages.append({"role": "tool", "tool_call_id": tc.id, "content": output})

In [ ]:
from strands import Agent, tool

@tool
def bash(cmd: str) -> str:
    """Run a bash command and returns results"""
    return subprocess.run(["bash", "-lc", cmd], capture_output=True)

Agent(tools=[bash])(f"list all files in this dir")